## Exploratory Data Analysis for the couple text-message insights

### TODO:
- Split the input data into dataframes of text-only, stickers-only, etc.

In [ ]:
import pandas as pd
import nltk
import json

%matplotlib inline

In [ ]:
with open('result.json') as f:
    chatlog_raw = json.load(f)


In [ ]:
df_chatlog = pd.DataFrame(chatlog_raw['messages'])

In [ ]:
df_chatlog.info()

In [ ]:
df_chatlog.head()

In [ ]:
# Pre-processing

# for message timeline
df_chatlog['date'] = pd.to_datetime(df_chatlog['date'])

In [ ]:
df_chatlog.groupby(['from','type'])['id'].count()

## Get Timeline of messages

In [ ]:
def get_timeline(df,window):
    wdf = df.copy(deep=True)
    td = pd.Timedelta(str(window)+'d')
    wdf['_byweek'] = wdf['date'].map(lambda x: x - td)
    
    stk = wdf.groupby(['from', pd.Grouper(key='_byweek',freq='W-MON')]).count().reset_index()
    msg_timeline = stk.pivot(index='_byweek', columns='from',values='id')
    
    return msg_timeline

In [ ]:
get_timeline(df_chatlog,7).plot(figsize=(20,10),style='-x')

## Message Statistics 

### On a week to week basis

In [ ]:
um = get_timeline(df_chatlog,7)

In [ ]:
um.columns.name = None
um.index.name = None

In [ ]:
# On a week to week basis
um.describe()

### On a message by message basis

In [ ]:
from nltk.tokenize import RegexpTokenizer 
tokenizer = RegexpTokenizer('\w+|\$[\d\.]+|\S+')

import string
_punc = str.maketrans('', '', string.punctuation)

In [ ]:
def process_message(txt):
    _processed = []
    
    if type(txt) == list:
        txt = [item['text'] if type(item) == dict else item for item in txt]
        txt = ' '.join(txt)
    
    _tokens = tokenizer.tokenize(txt)
    for token in _tokens:
        token = token.translate(_punc)
        _lower = token.lower()
        if _lower == '':
            continue
        else:
            _processed.append(_lower)
            
    return _processed

df_chatlog['text_processed'] = df_chatlog['text'].map(lambda x: process_message(x))

In [ ]:
df_chatlog['text_processed'].head()

In [ ]:
msgs_df = df_chatlog[['from','text_processed']]

In [ ]:
msgs_df['num_words'] = msgs_df['text_processed'].map(lambda x: len(x))

In [ ]:
msg_stats = msgs_df.groupby(['from'])['num_words'].describe()

In [ ]:
msgs_df.sort_values(by='num_words',ascending=False)

In [ ]:
msgs_df.max()

In [ ]:
msg_stats

In [ ]:
# TODO: Find a way to plot message stats